In [ ]:
import torch
torch.set_default_dtype(torch.float64)

In [ ]:
from collections import deque

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tdg
from tqdm.notebook import tqdm

In [ ]:
NX = (7, 9, 11, 13, 15, 17, 19, 21, 23, 25,)
beta = 3./49
mu = 0
h = torch.tensor([0., 0., 0.])

In [ ]:
ensembles = deque()

In [ ]:
for nx in tqdm(NX):
    L = tdg.Lattice(nx)
    spacetime = tdg.Spacetime(2, L)
    
    S = tdg.Action(
        spacetime,
        tdg.Potential(-0.000000000000000001*tdg.LegoSphere([0,0])),
        beta=beta,
        mu=mu,
        h=h,
        )
    
    zero_configuration = spacetime.vector()
    cfgs = torch.stack((zero_configuration,))
    
    free = tdg.ensemble.GrandCanonical(S).from_configurations(cfgs).bootstrapped(1)
    ensembles.append(free)

In [ ]:
fig, ax = plt.subplots(11,1, sharex='col', figsize=(8,30))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx

    ax[0].plot((1/nx**2,), (e.N[0].real,        ), label=f'{nx}', marker='o')
    ax[1].plot((1/nx**2,), (e.T_by_TF[0].real,  ), label=f'{nx}', marker='o')

    ax[2].plot((1/nx**2,), (e.kinetic_by_kF4[0].real,        ), label=f'{nx}', marker='o')
    ax[3].plot((1/nx**2,), (e.internalEnergy_by_kF4[0].real,  ), label=f'{nx}', marker='o')
    
    ax[4].plot(((1/nx)**2,), (e.n_momentum[0,0].real,  ), label=f'{nx}', marker='o')
    
    ax[-6].plot((1/nx**2,), (e.w0_by_kF4[0].real, ), label=f'{nx}', marker='o')
    ax[-5].plot((1/nx**2,), (e.w2_by_kF2[0].real, ), label=f'{nx}', marker='o')
    ax[-4].plot((1/nx**2,), (e.w4[0].real,        ), label=f'{nx}', marker='o')
    
    ax[-3].plot((1/nx**2,), (e.b2_by_kF4[0].real,), label=f'{nx}', marker='o')
    ax[-2].plot((1/nx**2,), (e.b4_by_kF2[0].real,), label=f'{nx}', marker='o')
    ax[-1].plot((1/nx**2,), (e.b6       [0].real,), label=f'{nx}', marker='o')
    
ax[0].set_ylabel('N')
ax[1].set_ylabel('T/TF')

ax[2].set_ylabel('k / kF^4')
ax[3].set_ylabel('u / kF^4')

ax[4].set_ylabel('n(k=0)/L^2')

ax[-6].set_ylabel('w0 / kF^4')
ax[-5].set_ylabel('w2 / kF^2')
ax[-4].set_ylabel('w4')

ax[-3].set_ylabel('b2 / kF^4')
ax[-2].set_ylabel('b4 / kF^2')
ax[-1].set_ylabel('b6')


ax[-1].set_xlim([0, 0.022])
ax[-1].set_xlabel('1/nx^2')

# Momentum-dependent quantities

In [ ]:
fig, ax = plt.subplots(2,1, sharex='col', figsize=(8,12))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx
    nt = e.Action.Spacetime.nt
    k2 = L.linearize(L.ksq)

    ax[0].plot(e.momentum_by_kF_squared[0].real, e.n_momentum[0].real,        label=f'{nx=}', marker='o', linestyle='none', zorder=-nx)
    ax[1].plot(e.momentum_by_kF_squared[0].real, e.spin_momentum[0,:,2].real, label=f'{nx=}', marker='o', linestyle='none', zorder=-nx)


ax[0].set_yscale('log')
ax[1].set_yscale('symlog')
    
ax[0].set_ylabel('n(k) / L^2')
ax[1].set_ylabel('s(k) / L^2')

ax[0].set_xlabel('(k/kF)^2')
ax[0].legend(ncol=2)

# Condensate Fraction

In the free theory all the condensate fractions should be zero; a plot does not show anything more informative than a bunch of zeros.

In [ ]:
cutoff = 1e-10

for e in ensembles:
    print(
        e.Action.Spacetime.Lattice.nx,
        (e.pairing_singlet[0].abs() < cutoff).unique(return_counts=True),
        e.condensate_fraction_singlet[0],
        e.condensate_fraction_triplet_plus[0],
        e.condensate_fraction_triplet_zero[0],
        e.condensate_fraction_triplet_minus[0],
        e.condensate_fraction_up_down[0],
    )

In [ ]:
fig, AX = plt.subplots(4,5, sharex='col', sharey='row', figsize=(20,15))

for e in ensembles:
    
    L = e.Action.Spacetime.Lattice
    nx = L.nx
    k2 = L.linearize(L.ksq)

    for ax, EIGENVALUES in zip(AX, (
        e.pair_pair_eigenvalues_singlet, 
        e.pair_pair_eigenvalues_triplet_plus,
        e.pair_pair_eigenvalues_triplet_zero,
        e.pair_pair_eigenvalues_triplet_minus,
    )):
    
        for a, obs in zip(ax, (EIGENVALUES[0].flip(dims=(0,)))):# e.pair_pair_triplet_plus, e.pair_pair_triplet_zero, e.pair_pair_triplet_minus, e.pair_pair_up_down)):

            a.plot(((1/nx)**2,), (obs.real, ), label=f'{nx}', marker='o')

for (a, label) in zip(AX[:,0], ('singlet', 'triplet plus', 'triplet zero', 'triplet minus')):
    a.set_ylabel(label)

for (a, label) in zip(AX[0], ('largest eigenvalue', 'next largest', '...', 'and so on', '...')):
    a.set_title(label)

for a in ax:
    a.set_xlabel('(1 / nx)^2')
